<a href="https://colab.research.google.com/github/sipocz/_earthQuake/blob/main/_earthQuake_listfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:

#_Dengue

_PCVERSION_=False


if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"
else:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#--------------
def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")


def checkvalues(df,columnname,key):
    print(f"{columnname} ellenőrzése !")
    numok=0
    numerr=0
    for i in df.index:
        if df.at[i,columnname] in key:
            #print(df.at[i,columnname])
            numok+=1
            pass
        else:
            numerr+=1
            print(df.at[i,columnname],end=", ")
    sumall=numok+numerr
    print(f"\n{sumall} mintából {numerr} db nem volt megfelelő")


def createcolumn(df,columnname,keys):
    print(f"{columnname} cseréje megy")
    for key in keys:
        df[keys[key]]=0.0
    for key in keys:
        for i in df.index:
            if df.at[i,columnname]==key:
                df.at[i,keys[key]]=1.0


def create_dict(idx,list):
    o={}
    for i in list:
        o[i]=idx+"_"+i
    return o

def create_base_data(df):
    t=['n', 't', 'o']
    columnname="land_surface_condition"
    key=create_dict(columnname,t)


    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t= ['h', 'w', 'i', 'r', 'u']
    columnname="foundation_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['q', 'n', 'x']
    columnname="roof_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['z', 'v', 'f', 'm', 'x']
    columnname="ground_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['q', 's', 'j', 'x']
    columnname="other_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['j', 's', 't', 'o']
    columnname="position"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['c', 's', 'f', 'd', 'm', 'a', 'q', 'u', 'n', 'o']
    columnname="plan_configuration"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=['a', 'w', 'r', 'v']
    columnname="legal_ownership_status"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    df["geopos"]=df.geo_level_1_id+df.geo_level_2_id/10000+df.geo_level_3_id/1000000000
    return(df)



def kill_columns(df):
    notkey=["building_id","legal_ownership_status","geo_level_1_id",	"geo_level_2_id",	"geo_level_3_id", "land_surface_condition",	"foundation_type",	"roof_type",	"ground_floor_type",	"other_floor_type",	"position",	"plan_configuration"]
    for i in df.columns:
        if i in notkey:
            df.drop(columns=[i], inplace=True)
    return df

f=open(basedir+"/_EarthQuake/all_similarity.csv","a")
f.close()
X_train_ok=pd.read_csv(basedir+"/_EarthQuake/X_tran_ok.csv")
X_pred_ok=pd.read_csv(basedir+"/_EarthQuake/X_pred_ok.csv")
y_train_ok=pd.read_csv(basedir+"/_EarthQuake/y_train_ok.csv")

print(X_train_ok.head())


Mounted at /content/drive
   Unnamed: 0  count_floors_pre_eq  ...  legal_ownership_status_v     geopos
0           0                    2  ...                       1.0   6.048712
1           1                    2  ...                       1.0   8.090003
2           2                    2  ...                       1.0  21.036309
3           3                    2  ...                       1.0  22.041811
4           4                    3  ...                       1.0  11.013101

[5 rows x 67 columns]


In [62]:
X_train_ok.sort_values("geopos",inplace=True)
X_train_ok.reindex()
X_train_ok.reset_index(inplace=True)

X_pred_ok.sort_values("geopos",inplace=True)

X_pred_ok.reindex()

X_pred_ok.reset_index(inplace=True)






In [64]:
X_train_ok

,index,Unnamed: 0,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_t,land_surface_condition_o,foundation_type_h,foundation_type_w,foundation_type_i,foundation_type_r,foundation_type_u,roof_type_q,roof_type_n,roof_type_x,ground_floor_type_z,ground_floor_type_v,ground_floor_type_f,ground_floor_type_m,ground_floor_type_x,other_floor_type_q,other_floor_type_s,other_floor_type_j,other_floor_type_x,position_j,position_s,position_t,position_o,plan_configuration_c,plan_configuration_s,plan_configuration_f,plan_configuration_d,plan_configuration_m,plan_configuration_a,plan_configuration_q,plan_configuration_u,plan_configuration_n,plan_configuration_o,legal_ownership_status_a,legal_ownership_status_w,legal_ownership_status_r,legal_ownership_status_v,geopos
0,137998,137998,2,0,7,7,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006201
1,189003,189003,2,35,11,7,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006201
2,152965,152965,3,25,8,7,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006203
3,225551,225551,2,15,7,7,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006203
4,257260,257260,2,15,7,7,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.006203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,143106,143106,1,60,11,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.139111
260597,22206,22206,2,100,9,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.139111
260598,85769,85769,2,70,7,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.139111
260599,188328,188328,2,30,7,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.139111


In [74]:
st=0
en=3
old=0
for i in X_pred_ok.index:
    
    
    X_pre=X_pred_ok.loc[i]["geopos"]
    #print(f"i: {i:6} {X_pre} st: {st:8}, end: {en:8}")

    if old!=X_pre:
        print(f"i: {i:6} {X_pre} st: {st:8}, end: {en:8}")
        
        old=X_pre
        st=en
        
    while True: 
        X_tra=X_train_ok.loc[en]["geopos"]
        #print(f"X_tra: {X_tra:6}, {en}")
        en+=1
        if X_tra>X_pre:
            break
  
    
    #print(f" {X_pre} st: {st:8}, end: {en:8}")



i:      0 0.006201418 st:        0, end:        3
i:      1 0.006203149 st:        3, end:        4
i:      2 0.006203181 st:        4, end:        6
i:      4 0.006204065999999999 st:        6, end:       14
i:      5 0.006205354000000001 st:       14, end:       17
i:      6 0.006205578000000001 st:       17, end:       23
i:      8 0.006208165999999999 st:       23, end:       30
i:      9 0.0062120230000000005 st:       30, end:       43
i:     12 0.006901715 st:       43, end:       46
i:     14 0.00690386 st:       46, end:       48
i:     17 0.006904001 st:       48, end:       61
i:     21 0.006904184 st:       61, end:       72
i:     22 0.006904203 st:       72, end:       77
i:     26 0.0069058240000000005 st:       77, end:       90
i:     28 0.006905905 st:       90, end:      112
i:     29 0.006907944 st:      112, end:      116
i:     33 0.008100191999999999 st:      116, end:      124
i:     37 0.008102838999999999 st:      124, end:      130
i:     41 0.008106668000000

KeyboardInterrupt: ignored

In [ ]:
    
    for j in range(260000):
        newarr=X_train_ok.loc

        '''
        a=abs(X_train_ok.loc[j][-1]-X_pred_ok.loc[i][-1])
        if a <= min:
            print(f"Adatok: pred:{i:8}, trean:{j:7},{a:8}")
            print(f"{i},{j},{a:16}", file=f)
            min=a
            if a==0:
                db=db+1
            if db>10:
                exit
        '''